# Scrape

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Mars News

In [4]:
# Open browser to NASA Mars News Site
browser.visit('https://mars.nasa.gov/news/')

In [5]:
html = browser.html
soup = bs(html, 'html.parser')

# Search for news titles
title_results = soup.find_all('div', class_='content_title')

# Search for paragraph text under news titles
p_results = soup.find_all('div', class_='article_teaser_body')

# Extract first title and paragraph, and assign to variables
news_title = title_results[0].text
news_p = p_results[0].text

print(news_title)
print(news_p)

Space Samples Link NASA's Apollo 11 and Mars 2020
While separated by half a century, NASA's Apollo 11 and Mars 2020 missions share the same historic goal: returning samples to Earth.


### Mars Featured Imaged

In [6]:
# Open browser to JPL Featured Image
browser.visit('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')

In [7]:
# Click through to find full image
browser.click_link_by_partial_text('FULL IMAGE')

In [8]:
# Click again for full large image
time.sleep(2)
browser.click_link_by_partial_text('more info')

In [9]:
html = browser.html
soup = bs(html, 'html.parser')

# Search for image source
results = soup.find_all('figure', class_='lede')
relative_img_path = results[0].a['href']
featured_img = 'https://www.jpl.nasa.gov' + relative_img_path

print(featured_img)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16883_hires.jpg


### Mars Weather

In [10]:
# Open browser to Mars Weather Twitter Account
browser.visit('https://twitter.com/marswxreport?lang=en')

In [11]:
html = browser.html
soup = bs(html, 'html.parser')

# Search for tweets
results = soup.find_all('p', class_="TweetTextSize")

# Grab text of first tweet
mars_weather = results[0].text #strip line breaks and link

print(mars_weather)

InSight sol 251 (2019-08-11) low -101.0ºC (-149.7ºF) high -26.5ºC (-15.8ºF)
winds from the SSE at 4.1 m/s (9.2 mph) gusting to 17.5 m/s (39.1 mph)
pressure at 7.60 hPapic.twitter.com/9mgFzHl8t3


### Mars Facts

In [12]:
# Use Pandas to scrape data
tables = pd.read_html('https://space-facts.com/mars/')

# Take second table for Mars facts
df = tables[1]

# Rename columns and set index
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [13]:
# Convert table to html
df.to_html('table.html')

### Mars Hemispheres

In [14]:
# Open browser to USGS Astrogeology site
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

#### Search for hemisphere titles

In [15]:
html = browser.html
soup = bs(html, 'html.parser')

hemi_names = []

# Search for the names of all four hemispheres
results = soup.find_all('div', class_="collapsible results")
hemispheres = results[0].find_all('h3')

# Get text and store in list
for name in hemispheres:
    hemi_names.append(name.text)

hemi_names

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

#### Click through thumbnails to get full-sized hemisphere images

In [16]:
# Search for thumbnail links
thumbnail_results = results[0].find_all('a')
thumbnail_links = []

for thumbnail in thumbnail_results:
    
    # If the thumbnail element has an image...
    if (thumbnail.img):
        
        # then grab the attached link
        thumbnail_url = 'https://astrogeology.usgs.gov/' + thumbnail['href']
        
        # Append list with links
        thumbnail_links.append(thumbnail_url)

thumbnail_links

['https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced']

#### Extract image source of full-sized image

In [17]:
full_imgs = []

for url in thumbnail_links:
    
    # Click through each thumbanil link
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Scrape each page for the relative image path
    results = soup.find_all('img', class_='wide-image')
    relative_img_path = results[0]['src']
    
    # Combine the reltaive image path to get the full url
    img_link = 'https://astrogeology.usgs.gov/' + relative_img_path
    
    # Add full image links to a list
    full_imgs.append(img_link)

full_imgs

['https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg']

#### Store as a list of dictionaries

In [18]:
# Zip together the list of hemisphere names and hemisphere image links
mars_hemi_zip = zip(hemi_names, full_imgs)

hemisphere_image_urls = []

# Iterate through the zipped object
for title, img in mars_hemi_zip:
    
    mars_hemi_dict = {}
    
    # Add hemisphere title to dictionary
    mars_hemi_dict['title'] = title
    
    # Add image url to dictionary
    mars_hemi_dict['img_url'] = img
    
    # Append the list with dictionaries
    hemisphere_image_urls.append(mars_hemi_dict)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]